In [ ]:
# !rm -rf /kaggle/working/*

In [ ]:
# !python --version  
# Python 3.10.12

# Importing Libraries

In [ ]:
from typing import Optional, Union
from datetime import datetime

import os
import pickle
import gc
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter

!pip install /kaggle/input/pyspellchecker/pyspellchecker-0.8.0-py3-none-any.whl
from spellchecker import SpellChecker

import spacy

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
# from sklearn.preprocessing import MinMaxScaler

import seaborn as sns

# Globals

In [ ]:
SEED = 1337 # 45090448
VALIDATION_SHARE = 0.4
SAMPLE = True # if True then df.sample 
SAMPLE_RATE = 0.002
CSV_INPUT_PATH = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv'
INPUT_PATH = '/kaggle/input/'
OUTPUT_PATH = '/kaggle/working/'
MAX_ITER = 1200

In [ ]:
def printime():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

# Data Preparation

In [ ]:
def correct_text_spelling(text):
    spell = SpellChecker()
    words = re.findall(r'\b\w+\b', text)
    misspelled = spell.unknown(words)
    corrected_text = text
    for word in misspelled:
        if spell.correction(word):
            corrected_text = corrected_text.replace(word, spell.correction(word))
    return corrected_text

def correct_spelling(df, text_col: str = 'text') -> pd.DataFrame:
    df_ = df.copy()
    df_['corrected'] = df_[text_col].apply(correct_text_spelling)
    return df_['corrected']

In [ ]:
def get_df() -> pd.DataFrame:
    
    daigt = pd.read_csv(CSV_INPUT_PATH)
    daigt.rename(columns={'label': 'generated'}, inplace=True)
    df_ = daigt[daigt['RDizzl3_seven'] == True] # only True ones
    cols = ['text', 'generated']
#     df_['text'] = correct_spelling(df_, 'text')
    
    df_ = df_[cols]
    
    return df_

In [ ]:
def train_val_test_split(X: pd.DataFrame, y: pd.Series) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.Series, pd.Series, pd.Series]:
    X_tr, X_temp, y_tr, y_temp = train_test_split(X, y, test_size=VALIDATION_SHARE, stratify=y, random_state=SEED)
    X_val, X_val2, y_val, y_val2 = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=SEED)

    return X_tr, X_val, X_val2, y_tr, y_val, y_val2

In [ ]:
def get_corpus(df: pd.DataFrame, text_col: str = 'clean_text') -> list[str]:
    return df[text_col].to_list()

In [ ]:
def right_concat(df: pd.DataFrame, new_df: pd.DataFrame) -> pd.DataFrame:
    
    df_ = df.copy()
    new_df_ = new_df.copy()
    # forcing the index of the new datasets to match X_train and X_validation
    new_df_.set_index(df_.index, inplace=True)
    
    df_ = pd.concat([df_, new_df_], axis=1)
    
    return df_

def right_concat_split(X_tr: pd.DataFrame,  X_val: pd.DataFrame, 
                 new_df_tr: pd.DataFrame, new_df_val: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    
    See right_concat function def
    
    """
    return right_concat(X_tr, new_df_tr), right_concat(X_val, new_df_val)

# Features

## Text Cleaning and Features

In [ ]:
# cleaned text
def clean_text(df: pd.DataFrame) -> pd.Series:
    df_ = df.copy()
    
    df_['clean_text'] = (df_['text'].str.replace('\n\n', '') 
                                    .str.replace('\'s', '')
                                    .str.replace('[.,?!:;\'\\\\"]', '', regex=True)
                                    .str.lower()
                        )
    
    return df_['clean_text']

In [ ]:
def tfidf(corpus: list[str], vectorizer: Optional[TfidfVectorizer] = None) -> [pd.DataFrame | tuple[pd.DataFrame, pd.DataFrame]]:
    
    if vectorizer:
        M = vectorizer.transform(corpus)
        return M
    
    else:
        vectorizer = TfidfVectorizer(
                                     lowercase=False,
#                                      token_pattern = None,
                                     sublinear_tf=True,
                                     stop_words='english',
                                     ngram_range=(3,5)
                                    )
        
        M = vectorizer.fit_transform(corpus)
        
        return M, vectorizer 

In [ ]:
def pos_feature(df: pd.DataFrame, text_col: str = 'clean_text') -> pd.DataFrame:
    df_ = df.copy()
    
    nlp = spacy.load("en_core_web_sm")

    docs = [nlp(t) for t in df_[text_col]]
    pos_tags = [[token.pos_ for token in doc] for doc in docs]
    
    df_['pos_tags'] = pos_tags
    
    df_['pos_text'] = df_['pos_tags'].apply(lambda x: ' '.join(x))
    
    df_['NOUN_count'] = df_['pos_text'].str.count('NOUN')
    df_['VERB_count'] = df_['pos_text'].str.count('VERB')
    df_['ADJ_count'] = df_['pos_text'].str.count('ADJ')
    df_['ADV_count'] = df_['pos_text'].str.count('ADV')
    df_['ADP_count'] = df_['pos_text'].str.count('ADP')
    df_['PRON_count'] = df_['pos_text'].str.count('PRON')
    df_['PROPN_count'] = df_['pos_text'].str.count('PROPN')
    df_['PUNCT_count'] = df_['pos_text'].str.count('PUNCT')
    df_['AUX_count'] = df_['pos_text'].str.count('AUX')
    df_['NUM_count'] = df_['pos_text'].str.count('NUM')
    df_['X_count'] = df_['pos_text'].str.count('X')
    
    df_ = df_.drop(columns=['pos_tags', 'pos_text'])
    
    return df_
    
def stopwords_feature(df, text_col: str = 'clean_text') -> pd.DataFrame:
    df_ = df.copy()
    
    stop_words = set(stopwords.words('english'))

    def stopwords_counter_and_filter(row):
        words = word_tokenize(row[text_col])
        stopword_counter = sum(1 for w in words if w.lower() in stop_words)
        filtered_text = ' '.join(w for w in words if w.lower() not in stop_words)
        return stopword_counter, filtered_text

    df_[['number_of_stopwords', 'filtered_text']] = df_.apply(stopwords_counter_and_filter, axis=1, result_type='expand')

    return df_

def caps_to_periods_ratio_feature(df) -> pd.DataFrame:
    df_ = df.copy()
    
    parentheses_count = df_['parentheses_count']
    caps_count = df_['text'].str.count(r'[A-Z]')
    df_['caps_to_periods_ratio'] = caps_count / parentheses_count.replace(0, 1) # replace 0 by 1 to avoid deviding by 0
    
    return df_

## Feature Extraction

In [ ]:
def feature_extraction(df, text_col: str = 'clean_text') -> pd.DataFrame:
    """
    
    Extracts text based features for column text_col
    
    """
    
    df_ = df.copy()
    
    # text length
    df_['num_of_words'] = (df_[text_col].str.replace('\n\n', '') # removing row
                                .str.count(' ')+1)
    # uniqe words per row
    df_['text_vocab_size'] = df_[text_col].apply(lambda x : len(set(x.split())))
    
    df_['num_of_sentences'] = df_[text_col].str.count('\.')
    
    df_['parentheses_count'] = df_['text'].str.count('\(|\)')
    
    df_['semicolon_count'] = df_['text'].str.count(';')

    df_['hypen_count'] = df_['text'].str.count('-')
    
    df_['dash_count'] = df_['text'].str.count('—')
    
    df_['comma_count'] = df_['text'].str.count(',')
    
    df_['qm_count'] = df_['text'].str.count('\?')
    
    df_['en_count'] = df_['text'].str.count('!')
    
    df_['apostrophe_count'] = df_['text'].str.count('’')
    
    df_['paragraph_count'] = df_['text'].str.count('\n\n')
    
    # extract POS features
    df_ = pos_feature(df_, text_col)
    
    # df_ = stopwords_feature(df_, text_col)
    
    # count typos
#     df_ = typos_count_feature(df_)

    # calculate the ratio between the number of capital letters and periods
    df_ = caps_to_periods_ratio_feature(df_)
    
    return df_

# Evaluation

In [ ]:
def plt_cm(y_true, y_pred, set_type, model_name):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(3, 2))
    class_labels = ['Human', 'LLM']
    sns.heatmap(cm, annot=True, fmt="d", cmap="RdPu", cbar=True,
                xticklabels=class_labels, yticklabels=class_labels, ax=ax)
#     title_font = {'family': 'serif',
#                   'color': 'blue',
#                   'weight': 'bold',
#                   'size': 14
#                  }
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.set_title(f"{set_type}\nCM for {model_name}") # fontdict=title_font
    plt.show()
    
def plt_roc_auc(y_true, y_score, set_type, model_name):
    roc_auc_val = roc_auc_score(y_true, y_score)
    print(f"\nROC AUC for {model_name}: {roc_auc_val}\n")

    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(fpr, tpr, color='orchid', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.legend(loc="best")
    ax.set_title(f'{set_type}\nROC Curve for {model_name}')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    plt.show()

In [ ]:
# validation functions
def validation_analysis(y_tr, y_val, y_tr_pred, y_val_pred, model_name):
    
    print("\nTRAIN SET\n")
    
    print("Classification report:")
    print(classification_report(y_tr, y_tr_pred))
    
    plt_cm(y_tr, y_tr_pred, "Training Set", model_name)
    plt_roc_auc(y_tr, y_tr_pred, "Training Set", model_name)
    
    print("\nVALIDATION SET\n")
    
    print("Classification report:")
    print(classification_report(y_val, y_val_pred))
    
    plt_cm(y_val, y_val_pred, "Validation Set", model_name)
    plt_roc_auc(y_val, y_val_pred, "Validation Set", model_name)

# First Ensemble Layer

## Data Preparation

In [ ]:
def get_df_with_cleaned_text() -> pd.DataFrame:
    df = get_df()

    if SAMPLE:
        df = df.sample(frac=SAMPLE_RATE, random_state=SEED)
        
    df['clean_text'] = clean_text(df)

    return df

In [ ]:
def save_to_files(tr_pred, val_pred, val2_pred, y_tr, y_val, y_val2, ensemble, vectorizer, series_name, tfidf_type):
    tr_pred = pd.Series(tr_pred, index=y_tr.index, name=series_name)
    val_pred = pd.Series(val_pred, index=y_val.index, name=series_name)
    val2_pred = pd.Series(val2_pred, index=y_val2.index, name=series_name)
    
    # saving artifacts
    # saving processed data
    filepath = os.path.join(OUTPUT_PATH, f'{tfidf_type}_tfidf_tr_pred.csv')
    tr_pred.to_csv(filepath)
    
    filepath = os.path.join(OUTPUT_PATH, f'{tfidf_type}_tfidf_val_pred.csv')
    val_pred.to_csv(filepath)
    
    filepath = os.path.join(OUTPUT_PATH, f'{tfidf_type}_tfidf_val2_pred.csv')
    val2_pred.to_csv(filepath)
    
    # saving objects
    filepath = os.path.join(OUTPUT_PATH, f'{tfidf_type}_tfidf_ensemble.pickle')
    pickle.dump(ensemble, open(filepath, 'wb'))
    
    filepath = os.path.join(OUTPUT_PATH, f'{tfidf_type}_tfidf_vectorizer.pickle')
    pickle.dump(vectorizer, open(filepath, 'wb'))

## Training

In [ ]:
def get_vectorizer(X_tr, is_pos) -> tuple[pd.DataFrame, pd.DataFrame]:
    corpus_tr = get_corpus(X_tr, "pos_text" if is_pos else "clean_text")
    X_tr, vectorizer = tfidf(corpus_tr)
        
    return X_tr, vectorizer

In [ ]:
def get_ensemble() -> VotingClassifier:
    # training
    
    # svm_clf = SVC(random_state=SEED, verbose=False)
    lgbm_clf = LGBMClassifier(random_state=SEED, verbose=-1)
    cat_clf = CatBoostClassifier(random_seed=SEED, verbose=False)
    lr_clf = LogisticRegression(random_state=SEED, max_iter=MAX_ITER)
    sgd_clf = SGDClassifier(loss='log_loss', random_state=SEED, verbose=1)
    
    model_list = [
                    ('lgbm_clf',lgbm_clf), 
                    ('cat_clf', cat_clf),
                    ('lr_clf', lr_clf),
                    ('sgd_clf', sgd_clf)
                 ]
    
    ensemble = VotingClassifier(estimators=model_list, voting='soft', n_jobs=-1)
    
    return ensemble

In [ ]:
def get_ensemble_predict_and_proba(ensemble, tfidf_X_val, tfidf_X_tr):
    y_val_pred = ensemble.predict(tfidf_X_val)
    tfidf_tr_pred = ensemble.predict_proba(tfidf_X_tr)[:,1]
        
    return y_val_pred, tfidf_tr_pred

In [ ]:
def get_preds(X_val, df_X_val2, ensemble, vectorizer, is_pos) -> tuple[pd.Series, pd.Series]:
    # for the second stage of stack ensemble
    # train
    val_pred = ensemble.predict_proba(X_val)[:,1]
    
    # validation
    corpus_val2 = get_corpus(df_X_val2, "pos_text" if is_pos else "clean_text")
    X_val2 = tfidf(corpus_val2, vectorizer)
    val2_pred = ensemble.predict_proba(X_val2)[:,1]
    
    return val_pred, val2_pred

## Models by Feature Sets

In [ ]:
def text_tfidf_models():
    df = get_df_with_cleaned_text()

    # spliting the data
    X = df.drop(columns=['generated']).copy()
    y = df['generated'].copy()

    X_tr, X_val, X_val2, y_tr, y_val, y_val2 = train_val_test_split(X, y)
  
    tfidf_X_tr, vectorizer = get_vectorizer(X_tr, False)

    ensemble = get_ensemble()
    ensemble.fit(tfidf_X_tr, y_tr)

    y_tr_pred = ensemble.predict(tfidf_X_tr)

    # intial validation
    corpus_val = get_corpus(X_val)
    tfidf_X_val = tfidf(corpus_val, vectorizer)
    
    y_val_pred, tfidf_tr_pred = get_ensemble_predict_and_proba(ensemble, tfidf_X_val, tfidf_X_tr)

    validation_analysis(y_tr, y_val, y_tr_pred, y_val_pred, "TF-IDF Ensemble")
      
    tfidf_val_pred, tfidf_val2_pred = get_preds(tfidf_X_val, X_val2, ensemble, vectorizer, False)
    save_to_files(tfidf_tr_pred, tfidf_val_pred, tfidf_val2_pred, y_tr, y_val, y_val2, ensemble, vectorizer, "text_tfidf_pred", "text")
    
    return tfidf_tr_pred, tfidf_val_pred, tfidf_val2_pred, ensemble, vectorizer

In [ ]:
def pos_tfidf_models():
    df = get_df_with_cleaned_text()

    # extracting pos-related feature
    nlp = spacy.load("en_core_web_sm")

    docs = [nlp(t) for t in df['clean_text']]
    pos_tags = [[token.pos_ for token in doc] for doc in docs]
    
    df['pos_tags'] = pos_tags
    df['pos_text'] = df['pos_tags'].apply(lambda x: ' '.join(x))
    
    # spliting the data
    X = df.drop(columns=['generated']).copy()
    y = df['generated'].copy()

    X_tr, X_val, X_val2, y_tr, y_val, y_val2 = train_val_test_split(X, y)
    
    pos_tfidf_X_tr, vectorizer = get_vectorizer(X_tr, True)
    
    ensemble = get_ensemble()
    ensemble.fit(pos_tfidf_X_tr, y_tr)
    
    y_tr_pred = ensemble.predict(pos_tfidf_X_tr)

    # intial validation  
    pos_corpus_val = get_corpus(X_val, text_col='pos_text')
    pos_tfidf_X_val = tfidf(pos_corpus_val, vectorizer)
    
    y_val_pred, pos_tfidf_tr_pred = get_ensemble_predict_and_proba(ensemble, pos_tfidf_X_val, pos_tfidf_X_tr)

    validation_analysis(y_tr, y_val, y_tr_pred, y_val_pred ,"POS TF-IDF Ensemble")
    
    pos_tfidf_val_pred, pos_tfidf_val2_pred = get_preds(pos_tfidf_X_val, X_val2, ensemble, vectorizer, True)
    save_to_files(pos_tfidf_tr_pred, pos_tfidf_val_pred, pos_tfidf_val2_pred, y_tr, y_val, y_val2, ensemble, vectorizer, "pos_tfidf_pred", "pos")
    
    return pos_tfidf_tr_pred, pos_tfidf_val_pred, pos_tfidf_val2_pred, ensemble, vectorizer

In [ ]:
def numeric_features_models():
    df = get_df_with_cleaned_text()

    # extracting numeric features
    df = feature_extraction(df, 'clean_text')
    
    # spliting the data
    cols = ['text','clean_text', 'generated']
    X = df.drop(columns=cols).copy()
    y = df['generated'].copy()
    
    X_tr, X_val, X_val2, y_tr, y_val, y_val2 = train_val_test_split(X, y)
    
    num_ensemble = get_ensemble()

    num_ensemble.fit(X_tr, y_tr)
    
    y_tr_pred = num_ensemble.predict(X_tr)
    y_val_pred = num_ensemble.predict(X_val)
    
    # intial validation  
    validation_analysis(y_tr, y_val, y_tr_pred, y_val_pred ,"Numeric Ensemble")

    # for the second stage of stack ensemble
    # train
    num_val_pred = num_ensemble.predict_proba(X_val)[:,1]
    # validation
    num_val2_pred = num_ensemble.predict_proba(X_val2)[:,1]
    
    # transform to pd.Series 
    num_val_pred = pd.Series(num_val_pred, index=y_val.index, name='num_pred')
    num_val2_pred = pd.Series(num_val2_pred, index=y_val2.index, name='num_pred')
    
    # saving artifacts
    # saving processed data
    filepath = os.path.join(OUTPUT_PATH, 'num_val_pred.csv')
    num_val_pred.to_csv(filepath)
    
    filepath = os.path.join(OUTPUT_PATH, 'num_val2_pred.csv')
    num_val2_pred.to_csv(filepath)
    
    # saving objects
    filepath = os.path.join(OUTPUT_PATH, 'num_ensemble.pickle')
    pickle.dump(num_ensemble, open(filepath, 'wb'))
    
    return num_val_pred, num_val2_pred, num_ensemble

## Execution and Analysis

In [ ]:
printime()
tfidf_tr_pred, tfidf_val_pred, tfidf_val2_pred, tfidf_ensemble, tfidf_vectorizer = text_tfidf_models()
printime()
pos_tr_pred, pos_val_pred, pos_val2_pred, pos_ensemble, pos_vectorizer = pos_tfidf_models()
printime()
num_val_pred, num_val2_pred, num_ensemble = numeric_features_models()
printime()

# Second Ensemble Layer

In [ ]:
pred_val = pd.DataFrame({'tfidf_pred': tfidf_val_pred, 'pos_pred': pos_val_pred, 'num_pred': num_val_pred})
pred_val2 = pd.DataFrame({'tfidf_pred': tfidf_val2_pred, 'pos_pred': pos_val2_pred, 'num_pred': num_val2_pred})

# importing for intial y_tr, y_val
df = get_df()

if SAMPLE:
    df = df.sample(frac=SAMPLE_RATE, random_state=SEED)

X = df.copy()
y = df['generated'].copy()

del df
gc.collect()

# splitting the data
X_tr, X_val, X_val2, y_tr, y_val, y_val2 = train_val_test_split(X, y)

del X_tr, X_val, X_val2, y_tr
gc.collect()

cat_clf_stack = CatBoostClassifier(random_state=SEED, verbose=False)

cat_clf_stack.fit(pred_val, y_val)

y_val_pred = cat_clf_stack.predict(pred_val)
y_val2_pred = cat_clf_stack.predict(pred_val2)

filepath = os.path.join(OUTPUT_PATH, 'cat_clf_stack.pickle')
pickle.dump(cat_clf_stack, open(filepath, 'wb'))

del filepath, cat_clf_stack
gc.collect()

validation_analysis(y_val, y_val2, y_val_pred, y_val2_pred ,"CatBoost Over Predictions")

In [ ]:
printime()